In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

# Import the data
metric_data = pd.read_csv('/mnt/c/Users/nlemo/Desktop/FEBSim/TireSim/B1320run124.csv', sep=',')
metric_data

# Iterate over columns and modify column names
for col in metric_data.columns:
    metric_data.rename(columns={col: f"{col} {metric_data[col][0]}"}, inplace=True)

# Drop the first row since it's now serving as part of the headers
metric_data = metric_data.drop(0)

# Reset index
metric_data.reset_index(drop=True, inplace=True)

# Convert all values to floats
metric_data = metric_data.apply(pd.to_numeric, errors='coerce')

# Drop any rows with NaN values (if necessary)
metric_data = metric_data.dropna()

# Reset index
metric_data.reset_index(drop=True, inplace=True)
metric_data.columns

Index(['ET sec', 'V kph', 'N rpm', 'SA deg', 'IA deg', 'RL cm', 'RE cm',
       'P kPa', 'FX N', 'FY N', 'FZ N', 'MX N-m', 'MZ N-m', 'NFX none',
       'NFY none', 'RST deg C', 'TSTI deg C', 'TSTC deg C', 'TSTO deg C',
       'AMBTMP deg F', 'SR none'],
      dtype='object')

In [3]:
Q1 = metric_data[['SA deg', 'FY N']].quantile(0.25)
Q3 = metric_data[['SA deg', 'FY N']].quantile(0.75)
IQR = Q3 - Q1

# Apply the IQR filter for each specified column
#metric_data = metric_data[~((metric_data[['SA deg', 'FY N']] < (Q1 - 1.5 * IQR)) | (metric_data[['SA deg', 'FY N']] > (Q3 + 1.5 * IQR))).any(axis=1)]





# Define the Pacejka Magic Formula for lateral force (FY) with five coefficients
def pacejka_model(alpha, B, C, D, E, F):
    return D * np.sin(C * np.arctan(B * alpha - E * (B * alpha - np.arctan(B * alpha))) + F)

def fitted_curve(alpha, B_opt, C_opt, D_opt, E_opt, F_opt):
    return pacejka_model(alpha, B_opt, C_opt, D_opt, E_opt, F_opt)

# Define your bin edges based on the data range or desired intervals
pressure_bins = np.unique(np.round(metric_data['P kPa']))
camber_bins = np.unique(np.round(metric_data['IA deg']))
load_bins = np.unique(np.round(metric_data['FZ N']))

# Initialize empty lists to store binned data
binned_data = []

# Loop through each bin combination
for pressure in pressure_bins:
    for camber in camber_bins:
        for load in load_bins:
            # Filter data for current bin
            bin_filter = (np.round(metric_data['P kPa']) == pressure) & \
                         (np.round(metric_data['IA deg']) == camber) & \
                         (np.round(metric_data['FZ N']) == load)
            filtered_data = metric_data[bin_filter]

            if not filtered_data.empty:
                # Calculate mean for 'SA deg' and 'FY N' in the current bin
                mean_SA = filtered_data['SA deg'].mean()
                mean_FY = filtered_data['FY N'].mean()

                # Store the mean values along with the bin identifiers
                binned_data.append([pressure, camber, load, mean_SA, mean_FY])

# Convert binned data into a DataFrame
binned_df = pd.DataFrame(binned_data, columns=['Pressure', 'Camber', 'Load', 'Mean SA', 'Mean FY'])

# Fit the Pacejka model to the binned data
initial_guess = [0.1, 0, 10, 0, 0]  # Update as needed

#bounds = ([10, 10, 10, 10, 10],  # Lower bounds for B, C, D, E, F
#          [np.inf, np.inf, np.inf, np.inf, np.inf])

# Ensure there's enough data to fit
if not binned_df.empty and len(binned_df) > len(initial_guess):
    params_optimal, _ = curve_fit(pacejka_model, binned_df['Mean SA'], binned_df['Mean FY'],  p0=initial_guess, maxfev=150000)
    B_opt, C_opt, D_opt, E_opt, F_opt = params_optimal

    # Generate a range of slip angles (alpha) for plotting the fitted curve
    alpha_range = np.linspace(-10, 10, 500)

    # Calculate the fitted curve over the range of alpha
    fy_fitted = fitted_curve(alpha_range, *params_optimal)



NameError: name 'metric_data' is not defined

In [128]:

import plotly.graph_objects as go

r2 = r2_score(binned_df['Mean FY'], pacejka_model(binned_df['Mean SA'], *params_optimal))

# Plot the original binned data and the fitted curve using Plotly
fig = go.Figure()

# Add binned original data
fig.add_trace(go.Scatter(x=binned_df['Mean SA'], y=binned_df['Mean FY'], mode='markers', name='Binned Original Data'))

# Add fitted curve
fig.add_trace(go.Scatter(x=alpha_range, y=fy_fitted, mode='lines', name='Fitted Curve'))

# Update plot layout with R² in the title
fig.update_layout(title=f'Pacejka Model Fit to Binned Data (R² = {r2:.3f})',
                  xaxis_title='Slip Angle (SA deg)',
                  yaxis_title='Lateral Force (FY N)',
                  legend_title='Data Type')

# Show plot
fig.show()

# Print the optimized coefficients and the R² value
print("Optimized Coefficients:")
print("B:", B_opt)
print("C:", C_opt)
print("D:", D_opt)
print("E:", E_opt)
print("F:", F_opt)
print(f"R² Score: {r2:.3f}")



Optimized Coefficients:
B: 0.10644368491713986
C: -0.08585832067890116
D: 76266.59343013575
E: 2.9080798647997703
F: 3.738926947748882e-05
R² Score: 0.965


In [33]:
binned_df

,Pressure,Camber,Load,Mean SA,Mean FY
0,82.0,-5.0,-1122.0,-7.12800,2520.6100
1,82.0,-5.0,-1120.0,-7.14600,2552.3500
2,82.0,-5.0,-1116.0,-7.11100,2514.4900
3,82.0,-5.0,-1110.0,-7.05700,2533.1400
4,82.0,-5.0,-1108.0,-7.06175,2460.4775
...,...,...,...,...,...
2441,84.0,5.0,-1138.0,3.82700,-2353.3400
2442,84.0,5.0,-1136.0,4.61900,-2529.7200
2443,84.0,5.0,-1130.0,4.76500,-2523.6800
2444,84.0,5.0,-1129.0,4.61300,-2498.1200


In [23]:
metric_data = pd.read_csv('/mnt/c/Users/nlemo/Downloads/B1320run124.dat', sep='\t')
metric_data

,,,,,,,,,,,,,,,,,,,,TIRF Data File: Project 1320; Run 124
ET,V,N,SA,IA,RL,RE,P,FX,FY,FZ,MX,MZ,NFX,NFY,RST,TSTI,TSTC,TSTO,AMBTMP,SR
sec,kph,rpm,deg,deg,cm,cm,kPa,N,N,N,N-m,N-m,none,none,deg C,deg C,deg C,deg C,deg F,none
0.000,-0.04,0.30,-0.019,0.024,25.519,-31.100,82.81,0.94,-20.42,-1077.89,18.65,5.77,0.00,0.02,32.21,27.22,28.01,27.88,79.98,-1.821
0.020,-0.04,0.22,-0.015,0.030,25.521,-47.481,82.74,17.38,-21.23,-1079.06,18.81,10.63,-0.02,0.02,32.21,27.21,28.00,27.88,80.00,-1.538
0.040,-0.03,0.26,-0.012,0.028,25.518,-31.018,82.81,10.14,-17.73,-1079.13,19.41,7.42,-0.01,0.02,32.17,27.22,27.98,27.91,79.96,-1.823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447.660,0.72,8.86,-0.022,0.031,25.462,21.693,82.88,-29.20,45.34,-1113.58,55.07,-6.00,0.03,-0.04,35.12,45.14,46.74,51.62,80.39,0.174
447.680,0.68,9.56,-0.025,0.027,25.461,18.934,82.95,-36.27,41.95,-1111.59,54.10,-5.24,0.03,-0.04,34.61,45.14,46.73,51.65,80.36,0.345
447.700,0.66,9.84,-0.025,0.021,25.466,17.779,82.88,-21.38,40.62,-1114.95,54.52,-2.29,0.02,-0.04,35.71,45.20,46.73,51.63,80.39,0.432
447.720,0.62,9.33,-0.030,0.017,25.465,17.686,82.95,-39.38,41.43,-1116.28,55.24,-6.14,0.04,-0.04,36.22,45.22,46.71,51.68,80.37,0.440
